## Lists and Dictionaries

### Slice


- 实现了__getitem__和__setitem__方法的class的对象，都可以使用slice

- slice允许越界的index访问

- slice assignment没有长度相等的要求

In [2]:
a = ['a', 'b', 'c']
a[1:2] = [2, 3, 4] # length of slice assignments don't need to be the same
print(a)

['a', 2, 3, 4, 'c']


- stride和slice

stride： somelist[start:end:stride]

使用stride时不建议specify start和end, 可读性较差

如有需要，用一步slice单独实现。

In [16]:
temp = a[1:3] # instead of a[1:3:-1]
b = temp[::-1] # 用stride逆序的方法
print(b)

[3, 2]


- slice会发生shallow copy, 修改slicing的结果不会修改原list; 

  itertools的islice方法可以避免copy，用法与slice类似.
  
  islice(iterable, [start, ] stop [, step])
  
  只允许None或正数值。

In [14]:
import itertools

values = list(range(10))
first_five = list(itertools.islice(values, 5))
odds = list(itertools.islice(values, 1, None,2))
evens = list(itertools.islice(values, None, None, 2))
print('first five: ', first_five)
print('odds: ', odds)
print('evens: ', evens)

first five:  [0, 1, 2, 3, 4]
odds:  [1, 3, 5, 7, 9]
evens:  [0, 2, 4, 6, 8]


### Sort by Complex Criteria Using the `key` parameter

借助sort的key参数，灵活的使用sort

In [17]:
class Tool:
    def __init__(self, name, weight):
        self.name = name
        self.weight = weight
    
    def __repr__(self):
        return f'Tool({self.name}, {self.weight})'

tools = [
    Tool('hammer', 1.25), 
    Tool('screwdriver', 0.5),
    Tool('chisel', 0.25)    
]

tools.sort(key=lambda x: (x.name, x.weight), reverse=True) # do transformations on the values
'''
tuple自带了比较机制, 从左至右，同一位置元素相同则继续，不同就得到比较结果。
可以借助tuple实现multiple sorting criteria
'''
print(tools)

[Tool(screwdriver, 0.5), Tool(hammer, 1.25), Tool(chisel, 0.25)]


### Be Cautious When Relying on `dict` Insertion Ordering

python 3.5及之前的版本，dict的遍历顺序是随机的，与插入顺序无关。

但3.6开始，officially 3.7, "dict will preserve insertion order"

这意味着`keys`, `values`, `items`, `popitem`都会保序

In [19]:
class Student:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
a = Student('Bob', 12)
for key, value in a.__dict__.items(): # 按照声明顺序
    print(f'{key} = {value}')

name = Bob
age = 12
